# DATA PROCESSING AND CLEANUP

In [ ]:
pwd

In [ ]:
ls

In [ ]:
import os,tarfile

In [ ]:
# change current working directory to OULU
os.chdir('/media/pineapple45/Elements/anmol/btp/replayattack')

In [ ]:
# list all files in OULU


In [ ]:
# EXTRACT .tar FILES FOR TRAINING DATA
train_files = tarfile.open('replayattack-train.tar.gz')
train_files.extractall('train_files') # specify which folder to extract to
train_files.close()

In [ ]:
# EXTRACT .tar FILES FOR TESTING DATA
os.chdir('/media/pineapple45/Elements/anmol/btp/replayattack')
test_files = tarfile.open('replayattack-test.tar.gz')
os.chdir('/home/pineapple45/ANMOL/BTP/pixel-wise-supervision/replayattack')
test_files.extractall('test_files') # specify which folder to extract to
test_files.close()

In [ ]:
# EXTRACT .tar FILES FOR ENROLLMENT DATA
os.chdir('/media/pineapple45/Elements/anmol/btp/replayattack')
enrollment_files = tarfile.open('replayattack-enroll.tar.gz')
os.chdir('/home/pineapple45/ANMOL/BTP/pixel-wise-supervision/replayattack')
enrollment_files.extractall('enrollment_files') # specify which folder to extract to
enrollment_files.close()

In [ ]:
# EXTRACT .tar FILES FOR DEVLOPEMENT DATA
os.chdir('/media/pineapple45/Elements/anmol/btp/replayattack')
dev_files = tarfile.open('replayattack-devel.tar.gz')
os.chdir('/home/pineapple45/ANMOL/BTP/pixel-wise-supervision/replayattack')
dev_files.extractall('dev_files') # specify which folder to extract to
dev_files.close()


In [ ]:
# EXTRACT .tar FILES FOR FACE LOCATIONS DATA
os.chdir('/media/pineapple45/Elements/anmol/btp/replayattack')
face_loc_files = tarfile.open('replayattack-face-locations-v2.tar.gz')
os.chdir('/home/pineapple45/ANMOL/BTP/pixel-wise-supervision/replayattack')
face_loc_files.extractall('face_loc_files') # specify which folder to extract to
face_loc_files.close()


In [ ]:
# EXTRACT .tar FILES FOR PROTOCOLS DATA
os.chdir('/media/pineapple45/Elements/anmol/btp/replayattack')
protocols_files = tarfile.open('protocols-v3.tar.gz')
os.chdir('/home/pineapple45/ANMOL/BTP/pixel-wise-supervision/replayattack')
protocols_files.extractall('protocols_files') # specify which folder to extract to
protocols_files.close()


In [ ]:
# EXTRACT .tar FILES FOR COMPETETIONS ICB 2013 TESTSET DATA
os.chdir('/media/pineapple45/Elements/anmol/btp/replayattack')
icb_test_files = tarfile.open('competition_icb2013_testset.tar.gz')
os.chdir('/home/pineapple45/ANMOL/BTP/pixel-wise-supervision/replayattack')
icb_test_files.extractall('icb_test_files') # specify which folder to extract to
icb_test_files.close()


### EXTRACTING FRAMES FROM INPUT VIDEO

In [ ]:
import cv2
import numpy as np

In [ ]:
# DEFINING PARAMETERS FOR VIDEO CLASSIFICATION
IMG_SIZE=256
CHANNELS=3

In [ ]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
import cv2  
  
# Load the cascade  
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')  
  
# To capture video from existing video.   
cap = cv2.VideoCapture('test.mov')  
  
while True:  
    # Read the frame  
    _, img = cap.read()  
  
    # Convert to grayscale  
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
  
    # Detect the faces  
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)  
  
    # Draw the rectangle around each face  
    for (x, y, w, h) in faces:  
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)  
  
    # Display  
    cv2.imshow('Video', img)  
  
    # Stop if escape key is pressed  
    k = cv2.waitKey(30) & 0xff  
    if k==27:  
        break  
          
# Release the VideoCapture object  
cap.release()  

# ACTUAL COMPUTATION

In [ ]:
import numpy as np
import zipfile, os, cv2, random
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import load_model,Model
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
import time
import matplotlib.pyplot as plt
import matplotlib
import requests
import base64
import os
import glob
from pathlib import Path
from io import BytesIO
matplotlib.rcParams['figure.figsize'] = (12,12)
matplotlib.rcParams['axes.grid'] = False

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


# DataLoader

In [ ]:
# RUN THIS CODE CELL IN GOOGLE-COLLAB
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# RUN THIS CODE CELL IN GOOGLE-COLLAB
%cd '/content/drive/MyDrive/BTP/replay-attack-processed'

In [ ]:
# RUN THIS CODE CELL IN GOOGLE-COLLAB
# IMG_SHAPE = (256, 256, 3)
archive = zipfile.ZipFile('/content/drive/MyDrive/BTP/replay-attack-processed/train.zip', 'r')
image_paths = [k for k in list(archive.namelist()) if '.png' in k]
print('Train: Total =',len(image_paths), ', Spoof =',len([path for path in image_paths if 'spoof' in path]), ', Real =', len([path for path in image_paths if 'real' in path]))

archive = zipfile.ZipFile('/content/drive/MyDrive/BTP/replay-attack-processed/val.zip', 'r')
image_paths = [k for k in list(archive.namelist()) if '.png' in k]
print('Val: Total =',len(image_paths), ', Spoof =',len([path for path in image_paths if 'spoof' in path]), ', Real =', len([path for path in image_paths if 'real' in path]))

archive = zipfile.ZipFile('/content/drive/MyDrive/BTP/replay-attack-processed/test.zip', 'r')
image_paths = [k for k in list(archive.namelist()) if '.png' in k]
print('Test: Total =',len(image_paths), ', Spoof =',len([path for path in image_paths if 'spoof' in path]), ', Real =', len([path for path in image_paths if 'real' in path]))

In [ ]:
# RUN THIS CODE CELL LOCALLY
# IMG_SHAPE = (256, 256, 3)
image_paths = [k for k in glob.iglob('./data/train' + '/**/*.jpg', recursive=True)]
print('Train: Total =',len(image_paths), ', Attack =',len([path for path in image_paths if 'attack' in path]), ', Real =', len([path for path in image_paths if 'real' in path]))

image_paths = [k for k in glob.iglob('./data/test' + '/**/*.jpg', recursive=True)]
print('Test: Total =',len(image_paths), ', Attack =',len([path for path in image_paths if 'attack' in path]), ', Real =', len([path for path in image_paths if 'real' in path]))

image_paths = [k for k in glob.iglob('./data/val' + '/**/*.jpg', recursive=True)]
print('Validation: Total =',len(image_paths), ', Attack =',len([path for path in image_paths if 'attack' in path]), ', Real =', len([path for path in image_paths if 'real' in path]))

Train: Total = 91705 , Attack = 69215 , Real = 22490
Test: Total = 121028 , Attack = 91799 , Real = 29229
Validation: Total = 90952 , Attack = 68467 , Real = 22485


In [ ]:
def new_py_function(func, inp, Tout, name=None):
  def wrapped_func(*flat_inp):
    reconstructed_inp = tf.nest.pack_sequence_as(inp, flat_inp,expand_composites=True)
    out = func(*reconstructed_inp)
    return tf.nest.flatten(out, expand_composites=True)
  flat_Tout = tf.nest.flatten(Tout, expand_composites=True)
  flat_out = tf.py_function(
      func=wrapped_func, 
      inp=tf.nest.flatten(inp, expand_composites=True),
      Tout=[_tensor_spec_to_dtype(v) for v in flat_Tout],
      name=name)
  spec_out = tf.nest.map_structure(_dtype_to_tensor_spec, Tout, 
                                   expand_composites=True)
  out = tf.nest.pack_sequence_as(spec_out, flat_out, expand_composites=True)
  return out

def _dtype_to_tensor_spec(v):
  return tf.TensorSpec(None, v) if isinstance(v, tf.dtypes.DType) else v

def _tensor_spec_to_dtype(v):
  return v.dtype if isinstance(v, tf.TensorSpec) else v

In [ ]:
# RUN THIS CODE CELL LOCALLY
class DataLoader:
  def __init__(self, dataset_path, batch_size=4, image_size=(256, 256), shuffle=True):
    self.images_paths = [k for k in glob.iglob(dataset_path + '/**/*.jpg', recursive=True)]
    random.shuffle(self.images_paths)
    self.length = len(self.images_paths)
    self.dim = image_size
    dataset = tf.data.Dataset.from_tensor_slices(self.images_paths).map(
            self.pad_map_fn, num_parallel_calls=3
          )
    if shuffle:
#       dataset = dataset.shuffle(buffer_size=100000, reshuffle_each_iteration=True)
      dataset = dataset.shuffle(buffer_size=5000, reshuffle_each_iteration=True)
    dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(buffer_size=3)
    # dataset = dataset.repeat()
    self.dataset = dataset.apply(tf.data.experimental.ignore_errors())  
                  
  def __len__(self):
        return self.length

  def load_tf_image(self, image_path):
    image_path = image_path.numpy().decode("utf-8")
#     img_data = self.archive.read(image_path.strip())
#     img_data = Image.open(image_path)
#     img_data = img_data.tobytes("xbm", "rgb")
#     image = np.frombuffer(img_data, np.uint8)
    image = cv2.imread(f"{image_path}")
#     image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, self.dim)
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    image = tf.image.per_image_standardization(image)
    image = tf.image.convert_image_dtype(image, tf.float32)

    # label - attack = 0, real = 1
    if 'attack' in image_path:
      label =  tf.constant([0])
      label_8 = tf.zeros_like(tf.random.uniform(shape=(8,8,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
      label_4 = tf.zeros_like(tf.random.uniform(shape=(4,4,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
      label_2 = tf.zeros_like(tf.random.uniform(shape=(2,2,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
      label_1 = tf.zeros_like(tf.random.uniform(shape=(1,1,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
    if 'real' in image_path:
      label = tf.constant([1])
      label_8 = tf.ones_like(tf.random.uniform(shape=(8,8,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
      label_4 = tf.ones_like(tf.random.uniform(shape=(4,4,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
      label_2 = tf.ones_like(tf.random.uniform(shape=(2,2,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
      label_1 = tf.ones_like(tf.random.uniform(shape=(1,1,1), minval=0, maxval=1, dtype=tf.dtypes.float32))

    return {'image': image, 'label': label, 'label_8': label_8, 'label_4': label_4, 'label_2': label_2, 'label_1': label_1}

  def pad_map_fn(self, img_path):
    return new_py_function(self.load_tf_image, inp=[img_path], Tout=({"image": tf.float32, "label": tf.int32, 'label_8': tf.float32, 'label_4': tf.float32, 'label_2': tf.float32, 'label_1': tf.float32}))

In [ ]:
# RUN THIS CODE CELL IN GOOGLE-COLLAB
class DataLoader:
  def __init__(self, dataset_path, batch_size=4, image_size=(256, 256), shuffle=True):
    self.archive = zipfile.ZipFile(dataset_path, 'r')
    self.images_paths = [k for k in list(self.archive.namelist()) if '.png' in k]
    random.shuffle(self.images_paths)
    self.length = len(self.images_paths)
    self.dim = image_size
    dataset = tf.data.Dataset.from_tensor_slices(self.images_paths).map(
            self.pad_map_fn, num_parallel_calls=3
          )
    if shuffle:
      dtaset = dataset.shuffle(buffer_size=1000, reshuffle_each_iteration=True)
    dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(buffer_size=3)
    # dataset = dataset.repeat()
    self.dataset = dataset.apply(tf.data.experimental.ignore_errors())  
                  
  def __len__(self):
        return self.length

  def load_tf_image(self, image_path):
    image_path = image_path.numpy().decode("utf-8")
    img_data = self.archive.read(image_path.strip())
    image = np.frombuffer(img_data, np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, self.dim)
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    image = tf.image.per_image_standardization(image)
    image = tf.image.convert_image_dtype(image, tf.float32)

    # label - spoof = 0, real = 1
    if 'spoof' in image_path:
      label =  tf.constant([0])
      label_8 = tf.zeros_like(tf.random.uniform(shape=(8,8,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
      label_4 = tf.zeros_like(tf.random.uniform(shape=(4,4,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
      label_2 = tf.zeros_like(tf.random.uniform(shape=(2,2,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
      label_1 = tf.zeros_like(tf.random.uniform(shape=(1,1,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
    if 'real' in image_path:
      label = tf.constant([1])
      label_8 = tf.ones_like(tf.random.uniform(shape=(8,8,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
      label_4 = tf.ones_like(tf.random.uniform(shape=(4,4,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
      label_2 = tf.ones_like(tf.random.uniform(shape=(2,2,1), minval=0, maxval=1, dtype=tf.dtypes.float32))
      label_1 = tf.ones_like(tf.random.uniform(shape=(1,1,1), minval=0, maxval=1, dtype=tf.dtypes.float32))

    return {'image': image, 'label': label, 'label_8': label_8, 'label_4': label_4, 'label_2': label_2, 'label_1': label_1}

  def pad_map_fn(self, img_path):
    return new_py_function(self.load_tf_image, inp=[img_path], Tout=({"image": tf.float32, "label": tf.int32, 'label_8': tf.float32, 'label_4': tf.float32, 'label_2': tf.float32, 'label_1': tf.float32}))

In [ ]:
# RUN THIS CODE CELL LOCALLY
loader = DataLoader('./data/train', batch_size=32,shuffle=False)

2022-04-10 20:33:53.066962: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-10 20:33:53.067846: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
# RUN THIS CODE CELL IN GOOGLE-COLLAB
loader = DataLoader('/content/drive/MyDrive/BTP/replay-attack-processed/train.zip', batch_size=32)

In [ ]:
# loader.load_tf_image('./data/train/real/frame_15.jpg')['label_1']

In [ ]:
# loader.pad_map_fn('./data/train/real/frame_15.jpg')

In [ ]:
dataset = loader.dataset.take(2)
# list(dataset.as_numpy_iterator())
a = list(dataset.as_numpy_iterator())
for a in list(dataset.as_numpy_iterator()):
  print(a['image'].shape, a['label'].shape, a['label_8'].shape, a['label_4'].shape, a['label_2'].shape, a['label_1'].shape)

2022-04-10 20:34:08.038337: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


(32, 256, 256, 3) (32, 1) (32, 8, 8, 1) (32, 4, 4, 1) (32, 2, 2, 1) (32, 1, 1, 1)
(32, 256, 256, 3) (32, 1) (32, 8, 8, 1) (32, 4, 4, 1) (32, 2, 2, 1) (32, 1, 1, 1)
